In [13]:
import os
import numpy as np
import subprocess
from tqdm.auto import tqdm
import pandas as pd
from joblib import Parallel, delayed
import random
import time

def scompatt(output):
    l = output.decode("utf-8") .split("\n")
    lambdas = [float(x) for x in l[4].split("LAMBDAS: ")[1].replace("[", "").replace("]", "").replace(" ", "").split(",")]
    accuracy = l[10].split("Test accuracy: ")[1].split(", ")[0]
    loss = l[11].split("Test loss: ")[1].split(", ")[0]
    precision = l[12].split("Test precision: ")[1].split(", ")[0]
    recall = l[13].split("Test recall: ")[1].split(", ")[0]
    return lambdas, float(accuracy) / 100, float(loss), float(precision), float(recall)

def single_run(dataset, epochs, lambdas):
    base_command_1 = "python3 -u node_class_full.py --data " + dataset + " --epoch " + str(epochs) + " "
    base_command_2 = " --layer 3 --w_att 0.001 --w_fc2 0.0 --w_fc1 0.001 --dropout 0.25 --lr_fc 0.01 --lr_att 0.01 --layer_norm 1 --dev 0 "
    command = base_command_1 + base_command_2 + " --lambdas " + lambdas
#     print(command)
    output = subprocess.check_output(command, shell=True)
    return scompatt(output)

def print_stuff(stuff):
    print("LAMBDAS:", stuff[0])
    print("ACCURACY:", stuff[1])
    print("PRECISION:", stuff[3])
    print("RECALL:", stuff[4])

In [ ]:
epochs = 100
datasets = ["cornell", "texas", "wisconsin"]#, "chameleon", "cora"]
all_lambdas = ["2,3", "2,4", "2,5", "3,4", "3,5", "4,5", "2,3,4", "2,3,5", "2,4,5", "3,4,5", "2,3,4,5"]

def f(dataset, epochs, lambdas):
    df = {}
    output = single_run(dataset, epochs, lambdas)
    df["acc"] = output[1]
    df["pre"] = output[3]
    df["rec"] = output[4]
    df["n_lambdas"] = lambdas
    df["lambdas"] = output[0]
#     print("-" * 50)
#     print("LAMBDAS:", lambdas)
#     print_stuff(output)
    return df

for dataset in datasets:
    r = Parallel(n_jobs=-1)(delayed(f)(dataset, epochs, lambdas) for lambdas in all_lambdas)
    results = {"acc": [], "pre": [], "rec": [], "lambdas": [], "n_lambdas":[]}
    for x in r:
        results["acc"].append(x["acc"])
        results["pre"].append(x["pre"])
        results["rec"].append(x["rec"])
        results["lambdas"].append(x["lambdas"])
        results["n_lambdas"].append(x["n_lambdas"])
    pd.DataFrame.from_dict(results).to_csv("results/mega-" + dataset + "-" + str(int(time.time())) + ".csv", index=False)